In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statistics import stdev
import numpy as np
from scipy import stats
import statsmodels
from statsmodels.sandbox.stats.multicomp import multipletests
from matplotlib import pyplot

import scipy.stats
import sklearn.linear_model
import sklearn.neighbors
from sklearn.naive_bayes import GaussianNB

import sklearn.ensemble
import time
import bp_preprocessing as preprocessing

In [2]:
dev = pd.read_csv("dev.csv")

In [3]:
dev = dev[['group','AK1C1',
 'TAU',
 '1433G',
 'SCUB1',
 'FMOD',
 'AMYP',
 'CRIS3',
 'MYDGF',
 'RARR2',
 'ATS8',
 'PGK1',
 '1433Z',
 'SV2A',
 'TRH',
 'GUAD',
 'HV69D',
 'CO7',
 'SERC']]

In [4]:
dev

,group,AK1C1,TAU,1433G,SCUB1,FMOD,AMYP,CRIS3,MYDGF,RARR2,ATS8,PGK1,1433Z,SV2A,TRH,GUAD,HV69D,CO7,SERC
0,AD_MCI,18.009949,18.421606,22.471329,19.499176,23.995281,20.762366,17.475345,21.729926,24.855334,18.819866,22.611179,22.769748,20.768264,14.948166,22.269739,19.087294,24.369242,21.597414
1,Healthy,18.795230,18.255636,21.840747,19.720331,24.253167,20.503768,18.806484,21.872982,25.262400,20.127048,22.404432,22.331013,20.442076,16.581877,21.154784,22.774641,24.419797,21.653803
2,Healthy,19.083557,19.129840,22.448146,19.344676,23.408567,20.256659,18.675593,21.908379,24.586243,19.693912,22.501874,22.657142,20.530374,17.860609,21.420884,23.181192,24.466082,21.175356
3,AD_MCI,18.108128,19.934621,22.636775,19.684006,24.460110,20.351661,15.599738,21.678495,25.387522,20.568765,22.723048,22.944981,20.327775,17.274123,21.917507,24.669925,24.514284,21.671200
4,Healthy,18.650715,18.557176,21.824173,19.585008,24.180328,20.276796,18.267226,21.830032,25.000770,18.694625,22.361950,22.031433,19.883247,17.417208,20.485582,24.348318,24.870784,21.443932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Healthy,19.123224,18.611889,21.705861,18.992121,24.638736,20.232877,18.509092,21.895392,25.793417,20.741529,22.376311,22.086880,20.821832,17.242076,20.781667,24.185051,25.093713,20.726307
234,Healthy,20.130518,18.380580,21.953330,19.903244,24.912696,20.371756,18.258177,21.607310,26.024950,19.789644,22.524107,22.184716,20.122928,18.159375,20.124358,23.729045,24.579697,20.855775
235,Healthy,17.928082,19.516288,22.107162,20.307207,23.900876,20.081079,18.201544,21.600041,24.887781,19.961729,22.094252,22.377506,20.803016,17.417667,21.175874,23.346395,24.761225,21.736080
236,Healthy,18.205782,18.081087,21.764129,19.689804,23.962881,20.814052,17.564997,22.016034,25.242071,20.439361,22.120500,22.134546,19.948511,16.731749,20.427924,25.587862,25.003600,21.338434


In [5]:
def rule_based_classifier(df):
    mean = np.mean(df.iloc[:,1:], axis=0)
    std = np.std(df.iloc[:,1:], axis=0)
    zscores = (df.iloc[:,1:] - mean) / std
 
    cols_1 = ["TAU"]
    cols_2 = ["1433G", "1433Z", "SERC"]
    cols_3 = ["GUAD", "PGK1", "AK1C1"]

    zscores_encoded = zscores
    for col in zscores.columns:
        if col in cols_1:
            zscores_encoded[col] = zscores[col]*4
        elif col in cols_2:
            zscores_encoded[col] = zscores[col]*3
        elif col in cols_3:
            zscores_encoded[col] = zscores[col]*2
        else:
            zscores_encoded[col] = zscores[col]*1

    zscores_encoded['Sum'] = zscores_encoded.sum(axis=1)
    zscores_encoded = pd.concat([df.iloc[:,:1], zscores_encoded], axis=1)
    return zscores_encoded

dev = rule_based_classifier(dev)

In [6]:
train, val = preprocessing.split_cats_by_tolerance(dev,tolerance=0.01,
                                                   silent=False,
                                                   randomstate=98281,
                                                   split=0.15,step=1,
                                                   categories=['Healthy','AD_MCI','PD','PD_MCI_LBD'])

{'Healthy': 112, 'AD_MCI': 37, 'PD_MCI_LBD': 27, 'PD': 26}
{'Healthy': 20, 'AD_MCI': 6, 'PD': 5, 'PD_MCI_LBD': 5}
Randstate: 98450

Percent Healthy in dev, test: [0.5544554455445545, 0.5555555555555556] 
Standard deviation of these values: 0.0005500550055005382 


Percent AD_MCI in dev, test: [0.18316831683168316, 0.16666666666666666] 
Standard deviation of these values: 0.008250825082508254 


Percent PD in dev, test: [0.12871287128712872, 0.1388888888888889] 
Standard deviation of these values: 0.0050880088008800894 


Percent PD_MCI_LBD in dev, test: [0.13366336633663367, 0.1388888888888889] 
Standard deviation of these values: 0.002612761276127612 



In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from xgboost import XGBClassifier

In [8]:
def apply_ml_model(dev, classifier):
    """
    finds the R2 score for different ML models
    """
    folds = sklearn.model_selection.LeaveOneOut().split(dev)

    fold_scores = []
    for train_indexes, val_indexes in folds: # the KFold splitter returns the indexes of the data, not the data itself
        train = dev.iloc[train_indexes]
        val = dev.iloc[val_indexes]

        scaler = sklearn.preprocessing.StandardScaler() # normalizing the numerical features
        train_X = scaler.fit_transform(train.iloc[:,1:]) 
        val_X = scaler.transform(val.iloc[:,1:])

        train_y = train.iloc[:,0] # 0th column is our target
        val_y = val.iloc[:,0]
        
        if classifier == "random_forest": 
            model = sklearn.ensemble.RandomForestClassifier()
        elif classifier == "naive_bayes":  
            model = sklearn.naive_bayes.GaussianNB()
        elif classifier == "decision_tree": 
            model = sklearn.tree.DecisionTreeClassifier()
        else: 
            print("wrong classifier named entered")
            
        model.fit(train_X, train_y) 
        fold_scores.append(
            sklearn.metrics.balanced_accuracy_score(val_y, model.predict(val_X)))
    return print(f"Leave-one-out cross validated balanced accuracy scores for {classifier} model (mean, std): ({np.mean(fold_scores)}, {np.std(fold_scores)})")

In [9]:
apply_ml_model(dev, classifier = 'random_forest')

Leave-one-out cross validated balanced accuracy scores for random_forest model (mean, std): (0.7016806722689075, 0.45752038909010534)
